In [1]:
import pandas as pd

In [2]:
polygon_pt_counts = pd.Series([1840, 1618, 1646, 1464, 1658, 1564, 1600, 1814, 1652, 1592, 1748, 1754, 1636, 1730,
                              1650, 1666, 1752, 2158, 1992, 1758, 1798])

In [4]:
polygon_pt_counts.describe()

count      21.000000
mean     1718.571429
std       151.573273
min      1464.000000
25%      1636.000000
50%      1666.000000
75%      1758.000000
max      2158.000000
dtype: float64

In [5]:
import csv

In [8]:
points_divided_df = pd.read_csv("../points_in_polys_validation.csv")

In [21]:
# PNTCNT is a unique identifier!
points_in_polys = {poly: points_divided_df[points_divided_df['PNTCNT'] == poly] 
                  for poly in points_divided_df['PNTCNT'].unique()}

In [28]:
samples = {poly: points_in_polys[poly].sample(n=10) for poly in points_in_polys}

In [35]:
samples = {poly: samples[poly][['x','y']] for poly in points_in_polys}

In [43]:
samples[1600]['y'][1229]

42.3516856583

In [56]:
import glob
import shutil
import random
for poly in samples:
    for _, sample in samples[poly].iterrows():
        # want to get rid of rounding error when these were truncated
        # but leave enough digits to find the photo
        for i in range(10):
            y = str(sample['y'])[:-i]
            matches = glob.glob("../data/streetphotos/42/" + y + "*")
            if len(matches) != 2:
                continue
            else:
                break

        if len(matches) != 2:
            print(sample)
            continue
        # choose left or right at random
        
        match = random.choice(matches)
        match_name = match.split('/')[-1]
        shutil.copyfile(match, "../data/streetphotos/validation_sample/" + match_name)
        
        

x   -71.165279
y    42.350975
Name: 2595, dtype: float64
x   -71.161673
y    42.353339
Name: 4110, dtype: float64
x   -71.158853
y    42.353638
Name: 7112, dtype: float64
x   -71.158926
y    42.354190
Name: 7016, dtype: float64
x   -71.151084
y    42.340326
Name: 8617, dtype: float64
x   -71.150516
y    42.353896
Name: 14109, dtype: float64
x   -71.150548
y    42.353844
Name: 14305, dtype: float64
x   -71.147010
y    42.354358
Name: 21109, dtype: float64
x   -71.147770
y    42.354036
Name: 20806, dtype: float64


In [85]:
validation_df = pd.read_csv("../data/streetphotos/validation_sample/validation_hand.csv", names=["label"])

In [86]:
validation_df.describe()

,label
count,200
unique,4
top,nn
freq,85


In [62]:
import os.path

In [74]:
sorted(list(os.walk("../data/streetphotos/validation_sample/"))[0][2])

['42.33336888249258,-71.15434008381818_left.jpg',
 '42.33356217596173,-71.1664137688052_left.jpg',
 '42.33381764166408,-71.16411340150124_right.jpg',
 '42.33393767008582,-71.15408701685091_right.jpg',
 '42.33400945441064,-71.16224555852912_right.jpg',
 '42.336201581096645,-71.15045532729124_left.jpg',
 '42.33627987039613,-71.15065697630948_right.jpg',
 '42.33634223370323,-71.14918461293868_left.jpg',
 '42.33643862627871,-71.16254879451807_right.jpg',
 '42.33755744991838,-71.15271796149143_left.jpg',
 '42.33763646510774,-71.16432911960659_left.jpg',
 '42.337966548274665,-71.1535147838182_right.jpg',
 '42.33840833596196,-71.15360927527627_right.jpg',
 '42.338923177416284,-71.14675696049002_left.jpg',
 '42.33908024075782,-71.14695120966272_right.jpg',
 '42.33912256300553,-71.15652097392953_left.jpg',
 '42.339195348076956,-71.16601665261385_right.jpg',
 '42.33942794216536,-71.15688682136036_right.jpg',
 '42.33960314472628,-71.14838152971132_right.jpg',
 '42.339631313065325,-71.158141140491

In [89]:
hand_predictions = {}
for file, pred in zip(sorted(list(os.walk("../data/streetphotos/validation_sample/"))[0][2])[:-1], validation_df['label']):
    hand_predictions[file] = pred

In [196]:
hand_predictions_df = pd.DataFrame.from_dict(hand_predictions, orient='index').reset_index()
hand_predictions_df['x'] = hand_predictions_df['index'].str.split(',').map(lambda x : float(x[0]))
hand_predictions_df['direction'] = hand_predictions_df['index'].str.split('_').map(lambda x : x[-1][:-4])
hand_predictions_df = hand_predictions_df[['x',0, 'direction']].set_index('x')

In [251]:
import pickle as pkl
with open("../data/streetphotos/all_pts_2_low_gamma.pkl", 'rb') as pklfile:
    all_pts_2_low_gamma = pkl.load(pklfile)
    
with open("../data/streetphotos/all_pts_1_low_gamma.pkl", 'rb') as pklfile:
    all_pts_1_low_gamma = pkl.load(pklfile)
    

In [253]:
all_pts_2_low_gamma['x'] = all_pts_2_low_gamma['point'].map(lambda x : x[1])
all_pts_1_low_gamma['x'] = all_pts_1_low_gamma['point'].map(lambda x : x[1])

In [254]:
all_pts_2_low_gamma = all_pts_2_low_gamma[['direction','x']].set_index('x')
all_pts_1_low_gamma = all_pts_1_low_gamma[['direction', 'x']].set_index('x')

In [256]:
maybe_1_low_gamma = hand_predictions_df.join(all_pts_1_low_gamma, how='inner', lsuffix="hand", rsuffix="cnn")
maybe_2_low_gamma = hand_predictions_df.join(all_pts_2_low_gamma, how='inner', lsuffix="hand", rsuffix="cnn")

In [257]:
len(hand_predictions_df[hand_predictions_df[0].isin(['s','o'])]), len(hand_predictions_df[hand_predictions_df[0].isin(['n'])]),len(hand_predictions_df[hand_predictions_df[0].isin(['nn'])])

(38, 77, 85)

In [225]:
len(hand_predictions_df[hand_predictions_df[0].isin(['s','o'])]), len(hand_predictions_df[hand_predictions_df[0].isin(['n'])])

(38, 77)

In [260]:
len(maybe_1_low_gamma[maybe_1_low_gamma[0].isin(['s','o']) & (maybe_1_low_gamma['directioncnn'] == maybe_1_low_gamma['directionhand'])])

24

In [262]:
len(maybe_1_low_gamma[maybe_1_low_gamma[0].isin(['n']) & (maybe_1_low_gamma['directioncnn'] == maybe_1_low_gamma['directionhand'])])

20

## For `maybe_1`

- the positive class has 38 members `len(hand_predictions_df[hand_predictions_df[0].isin(['s','o'])]`
- the negative class has 77 members `len(hand_predictions_df[hand_predictions_df[0].isin(['n'])])`
- 85 nulls `200 - 38 - 77`

Of the positive class

- 24 are correctly detected `len(maybe_1[maybe_1[0].isin(['s','o']) & (maybe_1['directioncnn'] == maybe_1['directionhand'])])`
- 14 are not `38 - 10`

Of the negative class
- 57 correctly detected
- 20 false positives `len(maybe_1[maybe_1[0].isin(['n']) & (maybe_1['directioncnn'] == maybe_1['directionhand'])])`

Recall: 63%
Precision: 54%

## For `maybe_2`
If we believe that this join is correct:

- the positive class has 39 members `len(maybe[maybe[0].isin(['s','o'])])`
- the actual negative class has 77 members `len(maybe[maybe[0] == 'n']`
- the remaining null photos have 85 members `len(maybe[maybe[0] == 'nn'])`

Of the positive class:
- 8 are correctly detected
- 31 are not

Of the negative class:
- 31 false positives `len(maybe[(pd.isnull(maybe['direction'])) & (maybe[0] != 'nn') & (maybe[0].isin(['s','o']))])`
- 46 correct

This gives us precision and recall of 21%.

Overall we see that the positive rate is `39/(39 + 77) * 100` or 33 percent.  The null rate is `1 - 22558/34294`, i.e. 34.2 percent.  

Precision: P(Pos | PosLabel). THIS DOES NOT CHANGE because we are conditioning on a positive label.

Recall: P(PosLabel | Pos) = P(PosLabel | Pos, NotNull)P(NotNull) + P(PosLabel | Pos, Null)P(Null)

P(PosLabel | Null) is zero, so

Recall = P(PosLabel | Pos, NotNull)P(NotNull) = 14%



In [229]:
len(all_pts_1)

3263